In [1]:
# import the necessary libraries
import os
from os import listdir
from PIL import Image
from numpy import asarray,expand_dims
from matplotlib import pyplot
from keras.models import load_model
import numpy as np
from keras_facenet import FaceNet
import pickle
import cv2
from mtcnn import MTCNN

In [2]:
# we are going to use harr cacade first 
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
# if harr cascade is unable to detect we will keep mtcnn for that case
# Initialize the MTCNN detector
mtcnn = MTCNN()
# we are going to use Facenet architecture for creating the embeddings from faces
model_face = FaceNet()

In [3]:
folder = 'training_images/'
# intialize empty dictionary in which we will store the embeddings with name of the person
database = {}

# iterate through all the images in the training images folder 
for filename in listdir(folder):
    path = folder + filename
    gbr1 = cv2.imread(folder + filename)
    
    # detect the face in the cropped photo :
    harr = HaarCascade.detectMultiScale(gbr1,1.1,4)
    faces_mtcnn = mtcnn.detect_faces(gbr1)
    # if the face is detected then get the width and height
    if len(harr)>0:
        x1, y1, width, height = harr[0]
    
    # if harr cascade is unable to detect the face use mtcnn 
    elif len(faces_mtcnn) > 0:
        x1, y1, width, height = faces_mtcnn[0]['box']

    # if no face is detected in the image just use the top left 10x10 pixels
    else:
        x1, y1, width, height = 1, 1, 10, 10
    
    
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    #convert from bgr to rgb
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)  # Convert from OpenCV to PIL
    # convert image as numpy array 
    gbr_array = asarray(gbr)

    # crop the face , resize it and store in face 
    face = gbr_array[y1:y2, x1:x2]
    face = Image.fromarray(face)
    face = face.resize((160, 160))
    face = asarray(face)

    # # resize the cv2 window to fit the face and draw a rectangle around the face 
    # cv2.namedWindow("output", cv2.WINDOW_NORMAL)       
    # cv2.resizeWindow("output", gbr1.shape[0],gbr1.shape[1])
    # cv2.rectangle(gbr1, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # # Show the image with the bounding box
    # cv2.imshow("output", gbr1)
    # # change the image when a key is pressed
    # cv2.waitKey(0)

    # facenet takes as input 4 dimensional array so we expand dimension 
    face = expand_dims(face, axis=0)
    signature = model_face.embeddings(face)

    # store the array in the database
    database[os.path.splitext(filename)[0]] = signature

cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━

In [4]:
# make a file named data_processed.pkl and store the database in it
myfile = open("data_processed.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()

In [6]:
# show the database
database